In [1]:
#imports
import ee
import requests
from datetime import datetime
from datetime import timedelta
from osgeo import ogr, osr
import math
from multiprocessing import JoinableQueue, Process
from queue import Queue
from threading import Thread
from os import path
from rtree.index import Index


# Run this notebook on Google Colab
for google drive integration

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/4AEj4KdMefRZHakr4BZccyoeBxRDp8P_FkamPx7o9OieiXOkx1REThE

Successfully saved authorization token.


In [4]:
import os
os.getcwd()

'/mnt/c/Users/georg/fyp'

# ge_download



In [6]:
def ge_download(inqueue, outqueue, errorqueue, params):
    start_date=params['start_date']
    scale=params['scale']
    end_date=params['end_date']
    while True:
        try:
            x, y, gadm_data, region= inqueue.get()
            img_id = '{0}_{1}'.format(x, y)
            tasklist=[]
            flag = True #replaced multi year download with placeholder flag
            while flag == True:
                region_to_download=ee.Geometry.Rectangle(region)
                ##use sentinel 2 image collection
                #filter for dates
                #filter images that intersect with region
                #select RGB bands
                #filter out excessively cloudy images
                imgc = ee.ImageCollection('COPERNICUS/S2_SR').\
                filterDate(start_date, end_date).\
                filterBounds(region_to_download).\
                select('B4', 'B3', 'B2').\
                filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))  

                im_list=imgc.toList(imgc.size())

                for xi in range(len(im_list.getInfo())):
                                    img = ee.Image(im_list.get(xi))
                                    img_dat=ee.Date(img.get('system:time_start')).format('YYYY_MM_dd').getInfo()
                                    img = img.divide(10000).visualize(bands =['B4', 'B3', 'B2'], min=0.0, max=0.3)
                                    name='{}_{}'.format(x,y)+'_S'+str(scale)

                                    task_config = {
                                      'description': name,
                                      'fileNamePrefix':name,
                                      'dimensions':60,    #max pixels of output image
                                      'region': region_to_download, #defines the region to be cropped and downloaded
                                      'folder': 's2_v2',  #directory to download images into
                                      'maxPixels' : 1e8  # upper bound for file size
                                      }

                                    task = ee.batch.Export.image.toDrive(img, **task_config)
                                    task.start()
                                    print('Processing',name,'task ID:',task.id)
                                    tasklist.append(task)              
                flag = False
                outqueue.put(img_id)

        except Exception as e:
            errorqueue.put('{}:{}'.format(img_id,str(e)))
            print(e)
        finally:
            inqueue.task_done()

# Geography


In [7]:
def coordToPixel(lng, lat, zoom=16):

    # Convert lat lng to Google "world" coordinate, then "pixel" coordinate.
    siny = math.sin(lat * math.pi / 180)
    siny = min(max(siny, -0.9999), 0.9999)

    worldlng = 256 * (0.5 + lng / 360)
    worldlat = 256 * (0.5 - math.log((1 + siny) / (1 - siny)) / (4 * math.pi))

    pixely = worldlat * math.pow(2, zoom)
    pixelx = worldlng * math.pow(2, zoom)

    return int(pixelx), int(pixely)


# Convert Google x-y pixel coordinates to WGS-84 long-lat coordinates.
def pixelToCoord(pixelx, pixely, zoom=16):

    # Convert x y to Google "world" coordinate, then wgs84 coordinate.
    worldlat = pixely / math.pow(2, zoom)
    worldlng = pixelx / math.pow(2, zoom)

    n = math.pi - 2 * math.pi * worldlat / 256
    lat = (180 / math.pi * math.atan(0.5 * (math.exp(n) - math.exp(-n))))

    lng = ((worldlng / 256) - 0.5) * 360

    return lng, lat

def save_info(inqueue, file_name):
    while True:
    # while not inqueue.empty():
        writer = open(file_name, "a", newline='\n')
        o = inqueue.get()
        # print("!!! ", str(o))
        writer.writelines(o)
        writer.writelines('\n')
        inqueue.task_done()
        writer.close()
    # file_to_add.close()

# shape_download_every_feature

In [8]:
def shape_download_RGB(ge_params, shapes=None, wd=None,filter_by_date=False,zoom=16,width=200, height=200):
    if wd==None:
        wd='/mnt/c/Users/georg/fyp'  #set working directory
    
    processed_file="{}processed.csv".format(wd) #make processed files log
    errors_file="{}errors.txt".format(wd)
    if not path.exists(processed_file):
        with open(processed_file,'w'):
            pass
    if not path.exists(errors_file):
        with open(errors_file,'w'):
            pass

    # Open layer and get extents in wgs84.
    idx = Index()
    ds = ogr.Open(shapes)
    lyr = ds.GetLayer(0)
    sr = lyr.GetSpatialRef()
    wgs84 = osr.SpatialReference()
    wgs84.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(sr, wgs84)
    need_transform = str(sr) != str(wgs84)
    geoms = {}

    lines = open("{}processed.csv".format(wd)).read().splitlines()
    processed = set(lines)

    # file_processed = open("processed.csv", "a", newline='\n')
    # errors_file_processed = open("processed.csv", "a", newline='\n')

    for count, feat in enumerate(lyr):
        if 'OBJECTID' in feat.keys():
            oid = feat.GetField('OBJECTID')
            date=feat.GetField('FDate')
        else:
            oid,date=None,None
        if filter_by_date:
            if date_threshold>datetime.strptime(date, '%Y/%m/%d').date():
                continue
        geom = feat.GetGeometryRef()
        if need_transform:
            geom.Transform(transform)
        envelope = geom.GetEnvelope()
        left, right, bottom, top = envelope
        # print(envelope)
        idx.insert(count, (left, bottom, right, top))

        geoms[count] = [oid, geom.Clone(), envelope, date]

    # Start queues, threads and processes.
    readingqueue = Queue()
    processingqueue = JoinableQueue()
    # processingqueue = Queue()
    errorqueue = JoinableQueue()
    writingqueue = JoinableQueue()
    # gadmqueue = JoinableQueue()

    processes=[]
    # process = Process(target=save_info, args=(processingqueue, processed_file))
    # process.daemon = True
    # process.start()
    # processes.append(process)
    #
    # process = Process(target=save_info, args=(errorqueue, errors_file))
    # process.daemon = True
    # process.start()
    # processes.append(process)

    for thread in range(1):
        querythread = Thread(target=ge_download, args=(readingqueue, processingqueue,errorqueue, ge_params))
        querythread.daemon = True
        querythread.start()

    skipped=0
    count=0
    cells=0

    for fid in geoms:
        oid, geom, envelope, date = geoms[fid]
        minx, maxx, miny, maxy = envelope

        # Convert min max coords to pixels (which start at upper left).
        pixelminx, pixelminy = coordToPixel(minx, maxy, zoom)
        pixelmaxx, pixelmaxy = coordToPixel(maxx, miny, zoom)

        # Round pixels to nearest "cell".
        pixelminx = int(math.floor(pixelminx / width) * width)
        pixelminy = int(math.floor(pixelminy / height) * height)
        pixelmaxx = int(math.ceil(pixelmaxx / width) * width)
        pixelmaxy = int(math.ceil(pixelmaxy / height) * height)

        x,y=pixelminx,pixelminy
        img_id = '{0}_{1}'.format(x, y)
        # if image_limit is not None and image_limit == count:
        #     break
        if img_id in processed:
            skipped+=1
        else:
            processed.add(img_id)

            # Get coordinates of image boundary.
            lr = pixelToCoord(pixelmaxx, pixelmaxy)
            ul = pixelToCoord(pixelminx, pixelminy)
            ll = [ul[0], lr[1]]
            ur = [lr[0], ul[1]]
            #
            #
            # gadm_data=[date]
            gadm_data=None
            count+=1
            readingqueue.put((x, y, gadm_data, [ul[0], ul[1], lr[0], lr[1]]))

    print(count)

    readingqueue.join()
    processingqueue.join()
    errorqueue.join()

    for process in processes:
        process.terminate()

    print(len(geoms))
    print('Count is', count)
    print('Skipped is', skipped)

# Main


In [ ]:
lights_params={'start_date':'2018-05-01',
               'end_date':'2018-09-30',
               'scale':10,
                }
shape_download_RGB(lights_params, shapes='/mnt/c/Users/georg/fyp/new_shapefile.shp', filter_by_date=False) #lights

11036
Processing 8286000_5497200_S10 task ID: 57ILMJXJYQ2JHKNFTIRZBKWE
Processing 8286000_5497200_S10 task ID: N6H5WQV366K3W3EFXU2WIAFA
Processing 8286000_5497200_S10 task ID: AFFNDTAJFZRICCCBL5TWBX5K
Processing 8286000_5497200_S10 task ID: 3ORN3X4EDWVCUJNBUDYKKTRE
Processing 8286000_5497200_S10 task ID: XN6PFYE4J36VSLTS3L6EQLFX
Processing 8286000_5497200_S10 task ID: J3YF4RHWN5GJO4N4TTIRJIIA
Processing 8286000_5497200_S10 task ID: CSPCZLNIE6KMJAO7ZOY5PRLT
Processing 8286000_5497200_S10 task ID: MLTWHNYUDBIUICWAG5TJJBI6
Processing 8286000_5497200_S10 task ID: SEGKLN27HLBO7SVTPTEAGBNB
Processing 8286000_5497200_S10 task ID: CZAEMOEQ4KQLDCBDAM6LULOA
Processing 8286000_5497200_S10 task ID: 3QRUVLKZ64QIV22CUMGLIUOA
Processing 8286000_5497200_S10 task ID: 5YHX3XL7R6AO2FOMHOU7UJLG
Processing 8286000_5497600_S10 task ID: Z6Y4SZETK3NEIFH2JRSIQKQJ
Processing 8286000_5497600_S10 task ID: Y3E76HGMU2N2IX3NEEJMAJ47
Processing 8286000_5497600_S10 task ID: ADYKULB5ATFK3PF6U7Z3QD5J
Processing 8286000_

Processing 8286800_5496800_S10 task ID: NGHOIWGDCR5DG5TXBC3NZKWL
Processing 8286800_5496800_S10 task ID: O7G5ONQSNOI3GYRNVXKZY2GB
Processing 8286800_5496800_S10 task ID: YRB6M4TPEUHVZR2WUCACMBJI
Processing 8286800_5496800_S10 task ID: 4G6JG2DBEDMFFN2BYJFGYBYM
Processing 8286800_5496800_S10 task ID: K3GPTMFVEOWEJNB5LI4O5STP
Processing 8286800_5496800_S10 task ID: Y3MSHPRERXJ5BYURVKCA2NHS
Processing 8286800_5497200_S10 task ID: 6S2UV7HWSDCST7CAB76SCRBF
Processing 8286800_5497200_S10 task ID: DQVJADSLM6ABDQVQO6GB763P
Processing 8286800_5497200_S10 task ID: 3MHVUUGI7MG2L2LFKHNOWW35
Processing 8286800_5497200_S10 task ID: D6RNQMFS5WF54ZLHKOJRJB3E
Processing 8286800_5497200_S10 task ID: QJDEPD6W2PFCIDEDRIT5VOBT
Processing 8286800_5497200_S10 task ID: CWJB2XIX7LC7BWCYQHKCZ6KK
Processing 8286800_5497200_S10 task ID: CTB5BFTZXKP3CTJBKJXBXPWY
Processing 8286800_5497200_S10 task ID: 3IE2Q2J4KAETX4A66WNLPJVK
Processing 8286800_5497200_S10 task ID: ULGBOQMOGNCXAN2FD4MHOFFW
Processing 8286800_549720

Processing 8287200_5497200_S10 task ID: CVD5SOYUNJK7BQXLX45RPL3S
Processing 8287200_5497200_S10 task ID: YPBWWJNOE54IIKKUIWD3YJS2
Processing 8287200_5497200_S10 task ID: PABIR2WG5DC3PHADUYMCUDSS
Processing 8287200_5497200_S10 task ID: MNDADJYMHPCB6HLQ3UTTVKCT
Processing 8287200_5497200_S10 task ID: DR36PPFDQDDZVPH623U3V5LF
Processing 8287200_5497200_S10 task ID: W3I5EGS4JOXKCXUGLNNJBMIM
Processing 8287200_5497200_S10 task ID: DL7DKXZVKT5KA7TZUG2SHSL4
Processing 8287200_5497200_S10 task ID: 2HKT5N5POY65FLUCMEMN72JG
Processing 8287200_5497200_S10 task ID: S7X4P4DXAN7EIELLQX3RK4FL
Processing 8287200_5497200_S10 task ID: DSJ6AHH222A2W6WVUILXPZ2V
Processing 8287200_5497200_S10 task ID: H67HN7ADYC4C6QRZYPFB23GV
Processing 8287200_5497600_S10 task ID: GFCGP2TXCQIHAWYSBDRREXHX
Processing 8287200_5497600_S10 task ID: FRX4VPWOJQNJZZYRULQIJCVC
Processing 8287200_5497600_S10 task ID: SQNNSDQIPOFXDDKJT6ZTHAJ2
Processing 8287200_5497600_S10 task ID: 43OOEKYROSN3AR476QZ5VGXJ
Processing 8287200_549760

Processing 8287600_5498000_S10 task ID: 7AYROWIUM3NHHRLQUFHCT6LP
Processing 8287600_5498000_S10 task ID: ZPWTSBYOZJFYU42J3GFJ33IU
Processing 8287600_5498000_S10 task ID: VLFZQFDLLQUVXQQEVLZAYKLD
Processing 8287600_5498000_S10 task ID: NHRVNWBPPRVHRANSUJVY2E2Y
Processing 8287600_5498400_S10 task ID: FR7QDJWFBTXVBNUEKO6P4WIV
Processing 8287600_5498400_S10 task ID: SLMUN2TZGDMYW3LYNYN2HROW
Processing 8287600_5498400_S10 task ID: XUO6MBKY5WARSEJUU4O73APW
Processing 8287600_5498400_S10 task ID: WP2S45HG25DNVYQ2JO4GLA4C
Processing 8287600_5498400_S10 task ID: T7IIHIVNJSHSL2ZG45QPBFXE
Processing 8287600_5498400_S10 task ID: 47EOM3GN4M5UBKA33W7YAQX3
Processing 8287600_5498400_S10 task ID: ALUUBHJJMAN3LCUNAQ2MJNDF
Processing 8287600_5498400_S10 task ID: S5ZQ4S3S7WOIGCEYODBBMUOS
Processing 8287600_5498400_S10 task ID: 5NMZF25TNP4ZFHX6G3XOIYF4
Processing 8287600_5498400_S10 task ID: 47XSLILTAB7RFZOHSYPFA5JR
Processing 8287600_5498400_S10 task ID: DYQFCUIWZH5TPD4YIVGK54SX
Processing 8287600_549840

Processing 8288000_5497600_S10 task ID: OLJU6V4Z5SZKGYA7QCZO7SGZ
Processing 8288000_5497600_S10 task ID: VVM2WX3ME56NM334R2IVJPRP
Processing 8288000_5497600_S10 task ID: U6DUVZY2PNCO2IZIZEVJT7J2
Processing 8288000_5497600_S10 task ID: 37MRO325RYKH6RXXIGSJUZBF
Processing 8288000_5497600_S10 task ID: ZRD56ZJW3REJXEE4VAOZMBDD
Processing 8288000_5497600_S10 task ID: HEDV3CLSNNZR5KLNRA4SDJ4X
Processing 8288000_5497600_S10 task ID: AHI3JOAQ3MDCMSM3XXTPHVZF
Processing 8288000_5497600_S10 task ID: 4GOXMGMQMLS2SLE2HSDUPAES
Processing 8288000_5497600_S10 task ID: U66BFP52C7VF26YTFFFWRJRN
Processing 8288000_5498000_S10 task ID: C7H7TRCPYBKZAZD3ONTOZGDF
Processing 8288000_5498000_S10 task ID: UVZYEBLELDE5MXTT3OU7DESK
Processing 8288000_5498000_S10 task ID: 3PRW7YDENL2HIGTAQDKE7SJC
Processing 8288000_5498000_S10 task ID: JLXN4VUWNOANJY4X7B5AYOUL
Processing 8288000_5498000_S10 task ID: G743ZWJMBVYRTL7W43K5UKVQ
Processing 8288000_5498000_S10 task ID: SES3YFGUKDWM7VS25J2WV6VO
Processing 8288000_549800

...

Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9602800_6074000_S10 task ID: 4N5B4Y5265NYI5UCTYRGVM4Y
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9602800_6076000_S10 task ID: 5OBB34HMIQLVAHZB2EGOVARJ
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9603200_6066800_S10 task ID: QRLIUOUTBP3ABXOX6JRVXZJK
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the q

Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9604000_6074400_S10 task ID: MQ5PMFOQ6Q3JJXMJYR6YKRJI
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9604000_6076000_S10 task ID: WJTFTXRUFQ64THLWEY6JTPRS
Processing 9604000_6076000_S10 task ID: SBABDXGC6AA4MBV23Q5AXDLN
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the q

Processing 9605200_6073200_S10 task ID: 5WEKD4NYRX3Y5MF6PYC2463Z
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9605200_6075000_S10 task ID: MEREW46UXVKER32XTBO72R4O
Processing 9605200_6075000_S10 task ID: M65HGI7LPCGZ5OJ66XU54COI
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9605200_6077200_S10 

Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9606800_6070800_S10 task ID: QC3AKU7BWQC2K2WHGV7JXAZA
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9606800_6071600_S10 task ID: 6X7PE4TTEI53VVDUFZFFWFMA
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9606800_6072400_S10 task ID: MESD3MRMVCEEXGEK7SOVNNVA
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9606800_6073600_S10 task ID: UNE5RGW4MCDTS6F22L247NQF
Too many tasks already in the queue (3000). Please w

Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9609200_6073600_S10 task ID: DC45WQC3QR5QM4BUUF46TRII
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9609600_6071600_S10 task ID: DDDWCUX3VDG6LYRWDVWNHFEH
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9609600_6072400_S10 task ID: 2CQ2AIFBVUB5HNZCM25CJ4CE
Too many tasks already in the queue (3000). Please wait for some of them to complete.
Processing 9609600_6072800_S10 task ID: XETJKBDGGHJY6ZBHIXUYCXFD
Too many tasks already in the queue (3000). Please w